In [2]:
# Import Packages
import torch
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torchvision.models as models
import torch.nn as nn
import torch.optim as optim
import numpy as np
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import os

In [3]:
# Convert all images to 100 x 100 images, and converts the image into tensors (numbers)
transformer = transforms.Compose([
    transforms.Resize(100),
    transforms.CenterCrop(100),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [4]:
# Load both datasets (training set and testing set)
train_set = datasets.ImageFolder("/media/mayi/Storage HDD/Summer Project/Brain tumor scans/Trainpath", transform = transformer)
test_set = datasets.ImageFolder("/media/mayi/Storage HDD/Summer Project/Brain tumor scans/Testpath", transform = transformer)

In [5]:
# Code to load data into engine (one for trainset and one for testset)
# batch_size = 32 means that there are 32 images going into the algorithm at a time
# shuffle = True means that the program will randomly go through each image throughout the directory
train_loader = torch.utils.data.DataLoader(train_set, batch_size = 32, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_set, batch_size =32, shuffle=True)

In [6]:
# This program uses a pretrained convulutional neural network (CNN), I'm just using this to make the process faster
model = models.densenet161(pretrained=True)

for parameter in model.parameters():
    parameter.requires_grad = False

In [7]:
# Number of labels are 2, (yes tumor and no tumor)
# Sequentially go through each layer of the neural network defined in the above cell
# Defines the layers of the neural network (ReLU, or "rectified linear" is the activation function for the layer)
classifier_input = model.classifier.in_features
num_labels = 2

classifier = nn.Sequential(nn.Linear(classifier_input, 500),
                           nn.ReLU(),
                           nn.Linear(500, 250),
                           nn.ReLU(),
                           nn.Linear(250, num_labels),
                           nn.LogSoftmax(dim=1))

model.classifier = classifier

In [8]:
# I'm not using a GPU, so all processes are within the CPU.
# If this program is being used by someone with a GPU, replace all instances of "cpu" with "cuda"
model.to("cpu")

DenseNet(
  (features): Sequential(
    (conv0): Conv2d(3, 96, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (norm0): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu0): ReLU(inplace=True)
    (pool0): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (denseblock1): _DenseBlock(
      (denselayer1): _DenseLayer(
        (norm1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inplace=True)
        (conv1): Conv2d(96, 192, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (norm2): BatchNorm2d(192, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu2): ReLU(inplace=True)
        (conv2): Conv2d(192, 48, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      )
      (denselayer2): _DenseLayer(
        (norm1): BatchNorm2d(144, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (rel

In [9]:
# Optimizer for loss (Adam)
error = nn.NLLLoss()
optimizer = optim.Adam(model.classifier.parameters())

In [11]:
# Go through the data 20 times (20 epochs) and optimize for loss
# Then print out the training/testing accuracy and each respective loss
# zero_grad() and loss.backward() are two things PyTorch needs in order to properly iterate through the set
epochs = 20
for epoch in range(epochs):
    train_loss = 0.0
    test_loss = 0.0
    accuracy = 0.0
    
    model.train()
    for inputs, labels in train_loader:
        inputs, labels = inputs.to("cpu"), labels.to("cpu")
        optimizer.zero_grad()
        output = model.forward(inputs)
        loss = error(output, labels)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()*inputs.size(0)

    model.eval()

    with torch.no_grad():
        for inputs, labels in test_loader:

            inputs, labels = inputs.to("cpu"), labels.to("cpu")
            output = model.forward(inputs)
            test_loss2 = error(output, labels)
            test_loss += test_loss2.item()*inputs.size(0)
            
            output = torch.exp(output)
            top_p, top_class = output.topk(1, dim=1)
            equals = top_class == labels.view(*top_class.shape)
            accuracy += torch.mean(equals.type(torch.FloatTensor)).item()
    
    train_loss = train_loss/len(train_loader.dataset)
    final_testing_loss = test_loss/len(test_loader.dataset)
    print('Accuracy: ', accuracy/len(test_loader))
    print("Epoch: " + str(epoch) + " Training Loss: " + str(train_loss) + " Testing Loss: " + str(final_testing_loss))

Accuracy:  0.8573545292019844
Epoch: 0 Training Loss: 0.46843343643331714 Testing Loss: 0.34208936842062726
Accuracy:  0.9050377160310745
Epoch: 1 Training Loss: 0.30029037577125867 Testing Loss: 0.21877516264265234
Accuracy:  0.953125
Epoch: 2 Training Loss: 0.17552919196988284 Testing Loss: 0.14012037582783832


KeyboardInterrupt: 

In [ ]:
# Puts data into evaluation mode to show predictions for test set
model.eval()

In [ ]:
# Function to process the image given
# Crops image to 100x100 once again
def process_image(image_path):
    img = Image.open(image_path)
    width, height = img.size
    img = img.resize((100, int(100*(height/width))) if width < height else (int(100*(width/height)), 100))
    width, height = img.size
    
    left = (width - 100)/2
    top = (height - 100)/2
    right = (width + 100)/2
    bottom = (height + 100)/2
    img = img.crop((left, top, right, bottom))
    
    img = np.array(img)
    
    img = img.transpose((2, 0, 1))
    
    img = img/255
    
    img[0] = (img[0] - 0.485)/0.229
    img[1] = (img[1] - 0.456)/0.224
    img[2] = (img[2] - 0.406)/0.225
    
    img = img[np.newaxis,:]
    
    image = torch.from_numpy(img)
    image = image.float()
    return image

# Function to predict the outcome of an input
def predict(image, model):
    output = model.forward(image)
    output = torch.exp(output)
    
    probs, classes = output.topk(1, dim=1)
    return probs.item(), classes.item()

# Simple matplotlib "plt.imshow()" function of the input given
def show_image(image):
    image = image.numpy()
    image[0] = image[0] * 0.226 + 0.445
    fig = plt.figure(figsize=(25, 4))
    plt.imshow(np.transpose(image[0], (1, 2, 0)))

In [ ]:
# This code allows the user to input an image path and the machine will give a prediction on
# whether the brain shown has a tumor or not, and also how sure the machine is about its prediction
image = process_image("/media/mayi/Storage HDD/Summer Project/Brain tumor scans/Predpath/jwpioej;ioaf.jpg")
top_prob, top_class = predict(image, model)
show_image(image)

if top_class == 0:
    result = "does not have a tumor."
else:
    result = "has a tumor"

print("The model is ", top_prob*100, "% certain that the brain in this image", result)

In [ ]:
# Same code as in the above cell, except this will iterate through a directory and list all predictions of
# all images in that directory.
directory = "/media/mayi/Storage HDD/Summer Project/Brain tumor scans/Predpath"
for filename in os.listdir(directory):
    if filename.endswith(".JPG") or filename.endswith(".jpg") or filename.endswith(".jpeg"): 
        endpoint = os.path.join(directory, filename)
        print("Found file in directory: " + endpoint)
        image = process_image(endpoint)
        top_prob, top_class = predict(image, model)
        show_image(image)

        if top_class == 0:
            result = "does not have a tumor."
        else:
            result = "has a tumor"

        print("The model is ", top_prob*100, "% certain that the brain in this image", result)